# Learning Python as an R user

# Import libraries

In [53]:
import pandas
import math
import numpy
import plotly.express
import plotly.offline
import scipy
import statsmodels.api
import statsmodels.formula.api
import patsy.contrasts

## Offline Plotting Settings

In [54]:
plotly.offline.init_notebook_mode(connected = True)

# Generate data
## `group` variable with 4 levels

In [55]:
group = numpy.repeat(["A", "B", "C", "D"], repeats = 25)

## Save parameters

In [56]:
mean = (3, 4, 4, 3)
sigma = (1, 1, 1, 1)
n = (25, 25, 25, 25)

## Random normal variable `x` whose means depend on levels of `group` variable

In [57]:
# Empty list for appending results
x = []

# Loop through equal-length mean, sigma, and n (group size)
for (i, j, k) in zip(mean, sigma, n):
    x.append(numpy.random.normal(loc = i, scale = j, size = k))

# Concatenate transforms the result into 1 array
x = numpy.concatenate(x)

## Variable `y` correlated r = 0.75 with x

In [58]:
y = x * 0.75 + numpy.random.normal(loc = 0, scale = 1, size = sum(n))

## Store variables in a data frame

In [59]:
data1 = pandas.DataFrame({"y": y, "x": x, "group": group})

## Create contrast variables for use in linear regression on `group` variable

In [60]:
# Helmert contrasts
group_helmert = patsy.contrasts.Helmert().code_without_intercept(list(set(group)))

# 2 main effects and 1 interaction contrast
group_factorial = patsy.contrasts.ContrastMatrix([[-1, -1, 1], [-1, 1, -1], [1, -1, -1], [1, 1, 1]], 
                                                 ["Main Effect 1", "Main Effect 2", "Interaction"])

## Add contrast variables to data frame

In [61]:
data1.loc[data1["group"] == "A", "main1"] = -1
data1.loc[data1["group"] == "B", "main1"] = -1
data1.loc[data1["group"] == "C", "main1"] = 1
data1.loc[data1["group"] == "D", "main1"] = 1

data1.loc[data1["group"] == "A", "main2"] = -1
data1.loc[data1["group"] == "B", "main2"] = 1
data1.loc[data1["group"] == "C", "main2"] = -1
data1.loc[data1["group"] == "D", "main2"] = 1

data1.loc[data1["group"] == "A", "interaction"] = 1
data1.loc[data1["group"] == "B", "interaction"] = -1
data1.loc[data1["group"] == "C", "interaction"] = -1
data1.loc[data1["group"] == "D", "interaction"] = 1

# Plots
## Boxplots

In [62]:
# boxplots of y by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "y")
)

In [63]:
# boxplots of x by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "x")
)

## Histograms

In [64]:
# y histograms by group
plotly.offline.iplot(
    plotly.express.histogram(data1, x = "y", facet_col = "group")
)

In [65]:
# x histograms by group
plotly.offline.iplot(
    plotly.express.histogram(data1, x = "x", facet_col = "group")
)

## Scatterplot

In [66]:
plotly.offline.iplot(
    plotly.express.scatter(data1, x = "x", y = "y", color = "group", trendline = "ols", facet_col = "group")
)

## Bars of `group` means with 95% confidence intervals

In [67]:
group_desc = data1.groupby("group")["y"].agg(["mean", "sem", "count"]).reset_index()
group_desc["df"] = group_desc["count"] - 1
group_desc["lower"] = group_desc["mean"] - scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]
group_desc["upper"] = group_desc["mean"] + scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]

# Plot
plotly.offline.iplot(
    plotly.express.bar(group_desc, x = "group", y = "mean", error_y_minus = "lower", error_y = "upper", color = "group")
)

# Descriptive Statistics

In [68]:
data1.groupby("group")[["x", "y"]].describe().round(2)

x                                               y                    \
      count  mean   std   min   25%   50%   75%   max count  mean   std   min   
group                                                                           
A      25.0  2.82  0.86  1.34  2.19  2.71  3.08  4.90  25.0  2.38  1.42 -0.06   
B      25.0  3.90  1.21  1.62  2.93  4.19  4.92  5.60  25.0  3.24  1.15  0.26   
C      25.0  3.87  1.09  2.03  3.23  3.77  4.44  6.26  25.0  2.89  1.58 -0.48   
D      25.0  2.96  1.01  1.34  2.26  2.88  3.64  5.27  25.0  2.60  1.15  0.56   

                               
        25%   50%   75%   max  
group                          
A      1.19  2.43  3.23  4.94  
B      2.56  3.49  3.98  5.04  
C      1.98  2.81  4.09  6.23  
D      1.51  2.83  3.55  4.70

# Correlation

In [69]:
# Save r and p-value
r1, pvalue1 = scipy.stats.pearsonr(x, y)

# Save degrees of freedom
ddf1 = len(x) - 2

# Compute t-statistic and degress of freedom, isf for upper tail of t distribution
t1 = scipy.stats.t.isf(pvalue1 / 2, df = ddf1)

# Compute standard error
se1 = r1 / t1

# Compute lower and upper confidence intervals
lower1, upper1 = (numpy.tanh(numpy.arctanh(r1) - 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)),
                  numpy.tanh(numpy.arctanh(r1) + 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)))

# Print most basic results
"r = {0}, 95%CI [{1}, {2}], t({3}) = {4}, p = {5}".format(r1.round(2), lower1.round(2), upper1.round(2), ddf1, t1.round(2), pvalue1.round(3))

'r = 0.6, 95%CI [0.46, 0.71], t(98) = 7.48, p = 0.0'

# Analyses
## Regression
### Fit linear regression

In [70]:
# regress y on numeric/continuous x
ols_fit1 = statsmodels.formula.api.ols("y ~ x", data = data1).fit()

### Results Summary

In [71]:
ols_fit1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     55.96
Date:                Mon, 13 Jan 2020   Prob (F-statistic):           3.19e-11
Time:                        15:23:31   Log-Likelihood:                -149.40
No. Observations:                 100   AIC:                             302.8
Df Residuals:                      98   BIC:                             308.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3655      0.341      1.073      0.286      -0.310       1.041
x              0.7125      0.095      7.481      0.000       0.523       0.902
==============================================================================
Omnibus:                        1.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.578   Jarque-Bera (JB):                1.029
Skew:                          -0.243   Prob(JB):                        0.598
Kurtosis:                       2.896   Cond. No.                         12.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [72]:
# helmert contrasts on group
ols_fit2 = statsmodels.formula.api.ols("y ~ C(group, group_helmert)", data = data1).fit()

### Results Summary

In [73]:
ols_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.943
Date:                Mon, 13 Jan 2020   Prob (F-statistic):              0.128
Time:                        15:23:31   Log-Likelihood:                -169.04
No. Observations:                 100   AIC:                             346.1
Df Residuals:                      96   BIC:                             356.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        2.7798      0.134     20.761      0.000       2.514       3.046
C(group, group_helmert)[H.B]     0.4303      0.189      2.272      0.025       0.054       0.806
C(group, group_helmert)[H.A]     0.0265      0.109      0.243      0.809      -0.190       0.244
C(group, group_helmert)[H.C]    -0.0602      0.077     -0.779      0.438      -0.214       0.093
==============================================================================
Omnibus:                        0.402   Durbin-Watson:                   2.269
Prob(Omnibus):                  0.818   Jarque-Bera (JB):                0.549
Skew:                          -0.124   Prob(JB):                        0.760
Kurtosis:                       2.735   Cond. No.                         2.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
# factorial contrasts (2 main effects and 1 interaction)
ols_fit3 = statsmodels.formula.api.ols("y ~ C(group, group_factorial)", data = data1).fit()

## Results Summary

In [75]:
ols_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.943
Date:                Mon, 13 Jan 2020   Prob (F-statistic):              0.128
Time:                        15:23:31   Log-Likelihood:                -169.04
No. Observations:                 100   AIC:                             346.1
Df Residuals:                      96   BIC:                             356.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  2.7798      0.134     20.761      0.000       2.514       3.046
C(group, group_factorial)Main Effect 1    -0.0337      0.134     -0.251      0.802      -0.299       0.232
C(group, group_factorial)Main Effect 2     0.1416      0.134      1.058      0.293      -0.124       0.407
C(group, group_factorial)Interaction      -0.2886      0.134     -2.156      0.034      -0.554      -0.023
==============================================================================
Omnibus:                        0.402   Durbin-Watson:                   2.269
Prob(Omnibus):                  0.818   Jarque-Bera (JB):                0.549
Skew:                          -0.124   Prob(JB):                        0.760
Kurtosis:                       2.735   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [76]:
ols_fit4 = statsmodels.formula.api.ols("y ~ main1 + main2 + interaction", data = data1).fit()

## Results Summary

In [77]:
ols_fit4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.943
Date:                Mon, 13 Jan 2020   Prob (F-statistic):              0.128
Time:                        15:23:31   Log-Likelihood:                -169.04
No. Observations:                 100   AIC:                             346.1
Df Residuals:                      96   BIC:                             356.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.7798      0.134     20.761      0.000       2.514       3.046
main1          -0.0337      0.134     -0.251      0.802      -0.299       0.232
main2           0.1416      0.134      1.058      0.293      -0.124       0.407
interaction    -0.2886      0.134     -2.156      0.034      -0.554      -0.023
==============================================================================
Omnibus:                        0.402   Durbin-Watson:                   2.269
Prob(Omnibus):                  0.818   Jarque-Bera (JB):                0.549
Skew:                          -0.124   Prob(JB):                        0.760
Kurtosis:                       2.735   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""